In [2]:
from IPython.display import clear_output
import sys

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import ee
import geemap
from google.colab import auth
auth.authenticate_user()

In [4]:
!rm -r /content/cerulean-cloud
branch = "feat/sss"
repo = "cerulean-cloud"
!git clone -b {branch} https://github.com/SkyTruth/{repo}.git
clear_output()

In [5]:
import geopandas as gpd
import json
import pandas as pd
from shapely import wkt

In [6]:
sys.path.append("/content/cerulean-cloud/")
from notebooks import sss

Downloaded GeoJSON to /Users/jonathanraphael/Downloads/3055322.geojson


DataSourceError: /Users/jonathanraphael/Downloads/3055322.geojson: No such file or directory

In [14]:
def create_ee_feature_for_infra(infra_gdf, scene_id, infra_index):
  """Creates an Earth Engine feature for a specific infrastructure instance.

  Args:
    infra_gdf: GeoDataFrame containing infrastructure data.
    scene_id: The ID of the scene.
    infra_index: The index of the infrastructure instance within the scene.

  Returns:
    An ee.Feature representing the specified infrastructure instance.
  """

  scene_infra = infra_gdf[(infra_gdf['scene_id'] == scene_id)]
  if len(scene_infra) > infra_index:
    infra_row = scene_infra.iloc[infra_index]
    lat,lon = infra_row['lat'], infra_row['lon']
    filename = infra_row['filename']
    if infra_row['geometry'].geom_type == 'Polygon':
      geometry = ee.Geometry.Polygon([list(infra_row['geometry'].exterior.coords)])
    elif infra_row['geometry'].geom_type == 'MultiPolygon':
      geometry = ee.Geometry.Polygon([list(polygon.exterior.coords) for polygon in infra_row['geometry'].geoms])
    properties = {'scene_id': infra_row['scene_id'], 'class': infra_row['class'], 'index': infra_index}
    return ee.Feature(geometry, properties), ee.Geometry.Point([float(lon), float(lat)]), filename
  else:
    return None


In [7]:
df = pd.read_csv('/content/drive/MyDrive/Cerulean Work/Cv3/groundtruth polys/infrastructure groundtruth/infrastructure_validation_points.csv')
df['geometry'] = df['geometry'].apply(wkt.loads)
infra_gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [15]:
# Example usage (assuming you want to display the first infrastructure instance for the first scene)

scene_id_to_display = scene_ids[0]
infra_index_to_display = 0

ee_feature, ee_point, filename = create_ee_feature_for_infra(infra_gdf, scene_id_to_display, infra_index_to_display)

image_id = f'COPERNICUS/S1_GRD/{scene_id_to_display}'
s1_image = ee.Image(image_id)
vis_params = {
    'min': -30,  # Adjust according to your data range
    'max': 0,
    'bands': ['VV'],  # Display the VV polarization band
    'palette': ['black','white']#, 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']  # Custom color palette
}



Map = geemap.Map()
Map.addLayer(s1_image, vis_params, 'Sentinel-1 VV')
Map.addLayer(ee_feature, {'color': 'red'}, 'Infra Slick')
Map.addLayer(ee_point, {'color': 'green'}, 'Infra Point')
Map.centerObject(ee_feature, 12)
Map

NameError: name 'scene_ids' is not defined